In [1]:
import psycopg2
import psycopg2.extras
import os
#import pygwalker as pyg

def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()
    
    
def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

In [38]:
query_filtro_data="SELECT *, CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) AS sortable_date_int FROM expresso_2023 WHERE centro = '8819009' AND CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) > 20231001;"
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = query_filtro_data
#query = "select * from expresso_2023 where centro='8819009'"
cur.execute(query)
results_expresso_ = cur.fetchall()


In [8]:
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = "select Cp7,longitude_x,latitude_y  from centroid_cp7_sig"
query_2 = "select cp7,point_x,point_y  from cp7coords"
cur.execute(query_2)
SIG_POSTAL_0 = cur.fetchall()

In [39]:
import pandas as pd
SIG_POSTAL_0=pd.DataFrame(SIG_POSTAL_0,columns=['cp7','longitude(X)','latitude(Y)'])
SIG_POSTAL_0['longitude(X)']= SIG_POSTAL_0['longitude(X)'].str.replace(",",".")
SIG_POSTAL_0['latitude(Y)']= SIG_POSTAL_0['latitude(Y)'].str.replace(",",".")
 
SIG_POSTAL_0 = SIG_POSTAL_0[SIG_POSTAL_0['longitude(X)'] != '']
SIG_POSTAL_0 = SIG_POSTAL_0[SIG_POSTAL_0['latitude(Y)'] != '']

SIG_POSTAL_0


,cp7,longitude(X),latitude(Y)
0,1000-001,-9.13868674899993,38.7330982610001
1,1000-002,-9.13936248099998,38.733887425
2,1000-003,-9.13994047099993,38.73459504
3,1000-004,-9.13864372599994,38.7331130320001
4,1000-005,-9.13953588199996,38.7341647800001
...,...,...,...
192038,9980-034,-31.115609638,39.673796849
192043,9980-039,-31.1123176689999,39.6723617820001
192045,9980-060,-31.111419481,39.6733136290001
192046,9980-070,-31.111683926,39.6729675230001


In [61]:
import pandas as pd
import numpy as np
df = pd.DataFrame(results_expresso_, columns=['data_criacao', 'centro', 'Giro', 'LOPTICA', 'JANELA_HORARIA', 'NOME', 'MORADA', 'CP', 'LOCALIDADE', 'COD_T_EVEN', 'DATA_EVENTO', 'LATITUDE', 'LONGITUDE','sortable_date_int'])
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]
#df=df[df['giro'].str[:3]==df['cp'].str[:3]]

# --. cálculo do centroid para cada ponto -- #


# -- considera apenas cp7 que ocorreram várias vezez--##

giro_unique_cp7 = df.groupby('cp')['data_criacao'].nunique().reset_index(name='N_Unique_cp7')

giro_unique_cp7 = giro_unique_cp7[giro_unique_cp7['N_Unique_cp7'] > 1]

giro_unique_cp7=np.array(giro_unique_cp7['cp'].unique())

df = df[df['cp'].isin(giro_unique_cp7)]

#-- excluir cp7 efetuados por várias rotas --##

unique_routes_by_cp = df.groupby('cp')['giro'].nunique().reset_index(name='Unique_Routes_Count')

unique_routes_by_cp = unique_routes_by_cp.sort_values(by='Unique_Routes_Count', ascending=False)

unique_routes_by_cp_filtered = unique_routes_by_cp[unique_routes_by_cp['Unique_Routes_Count'] ==1]

lista_excl_cp7=np.array(unique_routes_by_cp_filtered['cp'].unique())

df = df[df['cp'].isin(lista_excl_cp7)]

cp7_unique_counts = df['cp'].value_counts()

# -- cp7 para excluir quando em análise do cp4 em questão##

excl= ['2830-410','2830-411','2830-424','2830-429','2830-430','2830-432','2830-434','2830-435','2830-437',
'2830-438','2830-439','2830-441','2830-443','2830-444','2830-474','2830-476','2830-477','2830-478','2830-481','2830-581','2830-583',
'2830-592','2830-480','2830-513','2830-515','2830-519','2830-520','2830-440','2830-437','2830-066']

df = df[~df['cp'].isin(excl)]

#--- validação do giro --##

codigo_prestador='2860'
codigo_prestador_2=''
giros_excluir=['2970N','2970R','2970T','2970J','2970P']

giro_unique_counts = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_Unique_Data_Criacao')


giro_unique_counts = giro_unique_counts[
    ((giro_unique_counts['giro'].str[:4] == codigo_prestador) |
     (giro_unique_counts['giro'].str[:4] == codigo_prestador_2)) &
    (~giro_unique_counts['giro'].apply(lambda x: any(x.startswith(prefix) for prefix in giros_excluir)))
]



giro_unique_counts_average = giro_unique_counts['N_Unique_Data_Criacao'].mean()
giro_unique_counts_average
#giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > giro_unique_counts_average]
giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > 0]
giro_unique_counts

print(giro_unique_counts)

lista_val_giro=np.array(giro_unique_counts['giro'].unique())

print(lista_val_giro)

df = df[df['giro'].isin(lista_val_giro)]

#--#


# -- validação do cp4 -- 
unique_cp4_values = df['cp'].str[:4].unique()

cp_counts_by_unique_cp4 = {}
for cp4_value in unique_cp4_values:
    cp_counts_by_unique_cp4[cp4_value] = df[df['cp'].str[:4] == cp4_value]['cp'].count()

cp_counts = pd.DataFrame(list(cp_counts_by_unique_cp4.items()), columns=['CP4_Value', 'Count'])

cp_counts = cp_counts[cp_counts['Count'] > 200]  


cp_validation = np.array(cp_counts['CP4_Value'])

def expand_values(cp_values):
    expanded_values = []
    
    for value in cp_values:
        value = int(value)
        expanded_values.extend([str(value + i) for i in range(5)])
    
    return expanded_values

expanded_validation = expand_values(cp_validation)


print(cp_validation)
print(expanded_validation)

print(len(cp7_unique_counts))

     giro  N_Unique_Data_Criacao
48  2860A                     27
49  2860B                     32
50  2860D                     22
51  2860E                     28
['2860A' '2860B' '2860D' '2860E']
['2860']
['2860', '2861', '2862', '2863', '2864']
2743


In [431]:
excl= ['2830-410','2830-411','2830-424','2830-429','2830-430','2830-432','2830-434','2830-435','2830-437',
'2830-438','2830-439','2830-441','2830-443','2830-444','2830-474','2830-476','2830-477','2830-478','2830-481','2830-581','2830-583',
'2830-592','2830-480','2830-513','2830-515','2830-519','2830-520']



In [62]:
SIG_POSTAL = SIG_POSTAL_0['cp7'].str.replace(' ', '')
SIG_POSTAL = SIG_POSTAL_0[SIG_POSTAL_0['cp7'].str[:4].isin(expanded_validation)]
SIG_POSTAL = SIG_POSTAL[~SIG_POSTAL['cp7'].str[:8].isin(excl)]
SIG_POSTAL = SIG_POSTAL[~(SIG_POSTAL['longitude(X)'].isna() | (SIG_POSTAL['longitude(X)'] == ''))]
#SIG_POSTAL = SIG_POSTAL[SIG_POSTAL['cp7']=='2925-708']


SIG_POSTAL


,cp7,longitude(X),latitude(Y)
57744,2860-004,-9.03229601699996,38.6546646190001
57745,2860-005,-9.01467931799993,38.6524452200001
57746,2860-006,-9.00114172699995,38.6506550080001
57747,2860-007,-9.01873039099996,38.633671781
57748,2860-008,-9.00753982799995,38.640832249
...,...,...,...
58431,2864-004,-8.98712418799994,38.6595116340001
58432,2864-005,-8.98712418799994,38.6595116340001
58433,2864-006,-8.99126077399995,38.653835545
58434,2864-007,-8.99350716199996,38.6529858930001


In [70]:
from numpy import double

df_merge = pd.merge(df, SIG_POSTAL, left_on='cp', right_on='cp7', how='inner')

df_merge['latitude'] = pd.to_numeric(df_merge['latitude'], errors='coerce', downcast='float')
df_merge['longitude'] = pd.to_numeric(df_merge['longitude'], errors='coerce', downcast='float')
df_merge['latitude(Y)'] = pd.to_numeric(df_merge['latitude(Y)'], errors='coerce', downcast='float')
df_merge['longitude(X)'] = pd.to_numeric(df_merge['longitude(X)'], errors='coerce', downcast='float')

# Drop rows with NaN values in any of the specified columns
df_merge = df_merge.dropna(subset=['latitude', 'longitude', 'latitude(Y)', 'longitude(X)'])

# df_merge['latitude'] = df_merge['latitude'].astype(float)
# df_merge['longitude'] = df_merge['longitude'].astype(float)
# df_merge['latitude(Y)'] = df_merge['latitude(Y)'].astype(float)
# df_merge['longitude(X)'] = df_merge['longitude(X)'].astype(float)

print(len(df_merge))


1724


In [71]:
def calculate_distance(lon1, lat1, lon2, lat2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance


In [72]:
#df_merge['distance']=df_merge.apply(calculate_distance,axis=1)
df_merge['distance'] = calculate_distance(df_merge['latitude'], df_merge['longitude'], df_merge['latitude(Y)'], df_merge['longitude(X)'])
df_merge.head()

,data_criacao,centro,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,sortable_date_int,cp7,longitude(X),latitude(Y),distance
0,02/10/2023,8819009,2860A,DW697702677PT,09-14-S,CARLA SANTOS,"RUA DA QUINTINHA 6, 2DTO",2860-449,MOITA,I,02/10/2023 11:34:35,38.651176,-8.991366,20231002,2860-449,-8.991527,38.651005,0.025889
1,02/10/2023,8819009,2860A,DW698912218PT,09-19,FILIPE PEDRO,RUA DA QUINTINHA 6 2 DTO,2860-449,MOITA,I,02/10/2023 11:34:35,38.651176,-8.991366,20231002,2860-449,-8.991527,38.651005,0.025889
2,04/10/2023,8819009,2860A,FQ095860104PT,None,RUTE MOSCA,RUA DA QUINTINHA NUMERO 26A,2860-449,MOITA,I,04/10/2023 17:06:47,38.651157,-8.991665,20231004,2860-449,-8.991527,38.651005,0.022559
3,09/10/2023,8819009,2860A,DW699850024PT,09-19,JOSÉ ALMEIDA,RUA DA QUINTINHA N 28 1 DIREITO,2860-449,MOITA,I,09/10/2023 17:16:13,38.651443,-8.991134,20231009,2860-449,-8.991527,38.651005,0.064900
4,10/10/2023,8819009,2860A,DX244044898PT,09-19,CREMILDE SANTOS,"RUA DA QUINTINHA, N 9",2860-449,MOITA,I,10/10/2023 14:02:40,38.650887,-8.991372,20231010,2860-449,-8.991527,38.651005,0.021624


In [73]:
df=df_merge[df_merge.distance <500]
len(df)

1724

In [74]:
import pandas as pd
import numpy as np

#df.loc[:, 'N_Objetos'] = df.groupby(['giro', 'data_criacao'])['loptica'].transform('count')
df['N_Objetos'] = df.groupby(['giro', 'data_criacao'])['loptica'].transform('size')
df.loc[:, 'data_evento'] = pd.to_datetime(df['data_evento'], dayfirst=True)
df = df.sort_values(by=['giro', 'data_criacao', 'data_evento'], ascending=True)

df['cluster'] = pd.qcut(df.groupby(['giro', 'data_criacao']).cumcount(), 3, labels=[1, 2, 3])

df_aggregated = df.groupby(['cp']).agg({
    'giro': lambda x: x.value_counts().idxmax(),
    'cluster': lambda x: x.value_counts().idxmax(),
    'loptica': 'count',
    'longitude(X)': 'first',
    'latitude(Y)': 'first'
}).rename(columns={'loptica': 'count_loptica'}).reset_index()

# df_aggregated = df_aggregated[(df_aggregated['giro'] == '2835A')]

df_aggregated



,cp,giro,cluster,count_loptica,longitude(X),latitude(Y)
0,2860-009,2860B,2,27,-9.010311,38.655563
1,2860-010,2860B,2,31,-9.009361,38.656357
2,2860-027,2860B,3,10,-9.027989,38.654476
3,2860-028,2860B,3,2,-9.030747,38.656761
4,2860-030,2860B,3,9,-9.028591,38.655643
...,...,...,...,...,...,...
142,2860-998,2860B,3,8,-9.025126,38.653145
143,2864-001,2860A,1,2,-8.989426,38.648502
144,2864-002,2860A,1,24,-8.989426,38.648502
145,2864-006,2860A,2,6,-8.991261,38.653835


In [75]:

df_aggregated.loc[:,'count_loptica'] = df_aggregated['count_loptica'].astype(int)

df_aggregated.loc[:, 'giro'] = df_aggregated['giro'].astype(str)
df_aggregated.loc[:, 'cluster'] = df_aggregated['cluster'].astype(str)

df_aggregated['rota_zona'] = df_aggregated['giro'].str.cat(df_aggregated['cluster'])

df = df_aggregated.rename(columns={'cp': 'cp7', 'giro': 'rota', 'cluster': 'zona'})

df['cp4'] = df['cp7'].str[:4]

df.reset_index(drop=True, inplace=True)


df


,cp7,rota,zona,count_loptica,longitude(X),latitude(Y),rota_zona,cp4
0,2860-009,2860B,2,27,-9.010311,38.655563,2860B2,2860
1,2860-010,2860B,2,31,-9.009361,38.656357,2860B2,2860
2,2860-027,2860B,3,10,-9.027989,38.654476,2860B3,2860
3,2860-028,2860B,3,2,-9.030747,38.656761,2860B3,2860
4,2860-030,2860B,3,9,-9.028591,38.655643,2860B3,2860
...,...,...,...,...,...,...,...,...
142,2860-998,2860B,3,8,-9.025126,38.653145,2860B3,2860
143,2864-001,2860A,1,2,-8.989426,38.648502,2860A1,2864
144,2864-002,2860A,1,24,-8.989426,38.648502,2860A1,2864
145,2864-006,2860A,2,6,-8.991261,38.653835,2860A2,2864


In [76]:
from numpy import double

df_merge = pd.merge(df, SIG_POSTAL, left_on='cp7', right_on='cp7', how='right')
df_merge.drop(['longitude(X)_x', 'latitude(Y)_x'], axis=1, inplace=True)
df_merge = df_merge.rename(columns={'longitude(X)_y': 'longitude(X)', 'latitude(Y)_y': 'latitude(Y)'})
df_merge['latitude(Y)'] = df_merge['latitude(Y)'].astype(float)
df_merge['longitude(X)'] = df_merge['longitude(X)'].astype(float)
df=df_merge
df

,cp7,rota,zona,count_loptica,rota_zona,cp4,longitude(X),latitude(Y)
0,2860-004,NaN,NaN,NaN,NaN,NaN,-9.032296,38.654665
1,2860-005,NaN,NaN,NaN,NaN,NaN,-9.014679,38.652445
2,2860-006,NaN,NaN,NaN,NaN,NaN,-9.001142,38.650655
3,2860-007,NaN,NaN,NaN,NaN,NaN,-9.018730,38.633672
4,2860-008,NaN,NaN,NaN,NaN,NaN,-9.007540,38.640832
...,...,...,...,...,...,...,...,...
686,2864-004,NaN,NaN,NaN,NaN,NaN,-8.987124,38.659512
687,2864-005,NaN,NaN,NaN,NaN,NaN,-8.987124,38.659512
688,2864-006,2860A,2,6.0,2860A2,2864,-8.991261,38.653836
689,2864-007,2860A,1,5.0,2860A1,2864,-8.993507,38.652986


In [77]:
import os
from networkx import dfs_postorder_nodes
import pandas as pd
from scipy.spatial.distance import cdist
import pyproj
import numpy as np
from sklearn.cluster import KMeans


# create a transformer object to convert the coordinates to WGS84
transformer = pyproj.Transformer.from_crs(3763, 4326, always_xy=True)

# convert the POINT_X and POINT_Y columns to decimal degrees
df['longitude'], df['latitude'] = transformer.transform(df['longitude(X)'].tolist(), df['latitude(Y)'].tolist())

# get the X and Y coordinates and CP7 as arrays
coords = df[["longitude", "latitude"]].values
cp7 = df["cp7"].values.reshape(-1, 1)

# calculate the distance matrix using cdist
dist_matrix = cdist(coords, coords, "euclidean")

# set the diagonal to a high number to ignore the distance between a point and itself
dist_matrix[range(len(df)), range(len(df))] = float("inf")

# get the index of the minimum distance for each point
min_indexes = dist_matrix.argmin(axis=1)

# get the CP7 value corresponding to the minimum distance for each point
min_cp7 = cp7[min_indexes].squeeze()

# add the minimum distance and CP7 columns to the dataframe
df["min_distance"] = dist_matrix.min(axis=1)
df["min_cp7"] = min_cp7

#create a mask for blank rows in Giro, Zona, and Giro_Zona columns
mask = (df['rota'].isna()) & (df['zona'].isna()) & (df['rota_zona'].isna())

# fill the blank rows with the closest non-blank rows based on the CP7 values and their distances
for i in df[mask].index:
    min_distance = df.at[i, 'min_distance']
    closest_row_index = df[~mask].iloc[dist_matrix[i][~mask].argmin()].name
    for column in ['rota', 'zona', 'rota_zona']:
        df.at[i, column] = df.at[closest_row_index, column]
    df.at[i, 'min_distance'] = min_distance
    df.at[i, 'min_cp7'] = df.at[closest_row_index, 'cp7']


#dir_path = r"C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona"
#os.chdir(dir_path)


df_int=df.drop(['longitude', 'latitude', 'min_distance', 'min_cp7'],axis=1)
df_int.drop_duplicates(subset=['cp7'], keep='first', inplace=True)

#df_int.to_csv("Palmela_Completo.csv", index=False)

import pandas as pd
from sklearn.cluster import KMeans


columns = ['cp4', 'cp7', 'rota', 'zona', 'rota_zona', 'longitude(X)', 'latitude(Y)', 'longitude', 'latitude', 'min_distance', 'min_cp7']

df_final = pd.DataFrame(columns=columns)

#print(df_final)

for route in df['rota'].unique():
    # Filter the data frame to only include rows with the current Rota value
    df_route = df[df['rota'] == route]
    
    # Group the data frame by Rota_Zona and compute the centroid of each group
    #centroids = df_route.groupby(['Rota','Rota_Zona']).mean()[['longitude(X)', 'latitude(Y)']].reset_index()
    centroids = df_route.groupby(['rota', 'rota_zona']).mean(numeric_only=True)[['longitude(X)', 'latitude(Y)']].reset_index()
    centroids.columns = ['rota','rota_zona', 'centroid_X', 'centroid_Y']

    # Add a new column that specifies the centroid associated with each row
    df_route = pd.merge(df_route, centroids, on='rota_zona')

    # Use k-means to cluster the rows based on the longitude(X) and latitude(Y) columns
    kmeans = KMeans(n_clusters=len(centroids), init=centroids[['centroid_X', 'centroid_Y']], n_init=1)
    df_route['cluster'] = kmeans.fit_predict(df_route[['longitude(X)', 'latitude(Y)']]) % 3 + 1
    
    df_route = df_route.rename(columns={'rota_x': 'rota'})
    df_route = df_route.drop(['zona', 'rota_zona'], axis=1)
    df_route['zona'] = df_route['cluster'].astype(str)
    df_route['rota_zona'] = df_route['rota'].astype(str) + df_route['cluster'].astype(str)
    df_route = df_route[['cp4', 'cp7', 'rota', 'zona', 'rota_zona', 'longitude(X)', 'latitude(Y)', 'longitude', 'latitude', 'min_distance', 'min_cp7']]
    
    df_final = pd.concat([df_final, df_route], ignore_index=True)

    #df_final = df_final.append(df_route, ignore_index=True)
    

df_final=df_final.drop(['longitude', 'latitude', 'min_distance', 'min_cp7'],axis=1)
df_final.drop_duplicates(subset=['cp7'], keep='first', inplace=True)



df_final






/var/folders/97/743z88hx4pg224z9z13xy9100000gn/T/ipykernel_2447/2219260392.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df_route], ignore_index=True)


,cp4,cp7,rota,zona,rota_zona,longitude(X),latitude(Y)
0,NaN,2860-004,2860B,3,2860B3,-9.032296,38.654665
1,NaN,2860-011,2860B,3,2860B3,-9.024804,38.651245
2,NaN,2860-012,2860B,3,2860B3,-9.026342,38.650047
3,NaN,2860-015,2860B,3,2860B3,-9.026706,38.649525
4,NaN,2860-016,2860B,3,2860B3,-9.026909,38.648922
...,...,...,...,...,...,...,...
686,NaN,2860-497,2860A,3,2860A3,-8.994964,38.655525
687,NaN,2860-499,2860A,3,2860A3,-8.994330,38.654656
688,NaN,2860-638,2860A,3,2860A3,-8.996371,38.659894
689,NaN,2860-702,2860A,3,2860A3,-8.993629,38.658556


In [78]:
import folium
import re
import random  # for generating random colors

# Create a folium map centered at the mean latitude and longitude
m = folium.Map(location=[df_final['latitude(Y)'].mean(), df_final['longitude(X)'].mean()], zoom_start=12, tiles='OpenStreetMap')

# Create a color mapping dictionary
color_mapping = {}
unique_rotas = df_final['rota'].unique()

fixed_colors = [
    "#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD",
    "#34495E", "#229954", "#7D3C98", "#8E44AD", "#D35400",
    "#E74C3C", "#F39C12", "#2E86C1", "#A569BD", "#3498DB",
    "#27AE60", "#1ABC9C", "#E67E22", "#BDC3C7", "#34495E"
]

# Iterate through unique_rotas and assign fixed colors
for index, rota in enumerate(unique_rotas):
    color_mapping[rota] = fixed_colors[index % len(fixed_colors)]

# for rota in unique_rotas:
#     color_mapping[rota] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Create a feature group for each rota
rota_feature_groups = {}
for rota in unique_rotas:
    rota_feature_groups[rota] = folium.FeatureGroup(name=rota)  # Create a feature group for each rota

# Iterate through the rows of df_final
for _, row in df_final.iterrows():
    rota_color = color_mapping.get(row['rota'], '#1787FE')  # Default color if not found in mapping
    folium.RegularPolygonMarker(  # Use RegularPolygonMarker to change marker shape
        location=[row['latitude(Y)'], row['longitude(X)']],
        number_of_sides=10,  # Change this value to the number of sides you want (e.g., 3 for a triangle)
        radius=3,
        popup=re.sub(r'[^a-zA-Z]+', '', str(row['cp7'])),
        color=rota_color,
        fill=True,
        fill_color=rota_color
    ).add_to(rota_feature_groups[row['rota']])  # Add to the appropriate feature group

# Add each feature group to the map
for rota, feature_group in rota_feature_groups.items():
    m.add_child(feature_group)

# Add layer control to toggle feature groups
folium.LayerControl().add_to(m)

# Display the map
m


In [443]:
# one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona'

# if os.name == 'nt':  # Windows
#     one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\04.Dataset_CTT\01.CP7_Rota_Zona'
# else:  # Assuming Unix-like (Linux, macOS)
#     one_drive_folder = "/Users/rubenrocha/Library/CloudStorage/OneDrive-CTT-CorreiosdePortugal/04.Dataset_CTT/01.CP7_Rota_Zona"
#         # Construct the full file path

one_drive_folder = os.path.expanduser('~/Downloads')

# Combine the paths using os.path.join
desired_directory = os.path.join(one_drive_folder)

# Create the directory if it doesn't exist
os.makedirs(desired_directory, exist_ok=True)

# Define the file name
file_name = str(codigo_prestador)+".csv"

# Create the full file path
file_path = os.path.join(desired_directory, file_name)

# Save the DataFrame to the CSV file
df_final.to_csv(file_path, index=False)

print(f"CSV file saved to: {file_path}")

CSV file saved to: C:\Users\1420844/Downloads\2970.csv
